# Insert Into Cutomer

In [ ]:
import random
import string
from datetime import datetime, timedelta
import mysql.connector
from encription import Encryption

# Database connection details
DB_HOST = "localhost"
DB_USER = "root"
DB_PASSWORD = "1234"
DB_NAME = "computer_shop"

# Used phone numbers to ensure uniqueness
used_phones = set()

# Function to generate random customer data
def generate_customer_data():
    name = ''.join(random.choices(string.ascii_uppercase + string.ascii_lowercase, k=8))
    prefixes = [
        '010', '011', '012', '015', '016', '017', '018', '085', '086', '087', 
        '088', '089', '092', '093', '095', '096', '097', '098', '099'
    ]
    while True:
        phone = random.choice(prefixes) + ''.join(random.choices(string.digits, k=6))
        if phone not in used_phones:
            used_phones.add(phone)
            break

    start_date = datetime(2015, 1, 1)
    end_date = datetime.now()
    delta = end_date - start_date
    random_days = random.randint(0, delta.days)
    random_seconds = random.randint(0, 86399)  # seconds in a day
    reg_date = start_date + timedelta(days=random_days, seconds=random_seconds)

    # Format for MySQL DATETIME
    reg_date_str = reg_date.strftime('%Y-%m-%d %H:%M:%S')

    # login_date after reg_date, capped at now()
    login_date = reg_date + timedelta(days=random.randint(0, 365))
    login_date = min(login_date, datetime.now())
    login_date_str = login_date.strftime('%Y-%m-%d %H:%M:%S')

    img_path = "https://img.freepik.com/free-vector/blue-circle-with-white-user_78370-4707.jpg?semt=ais_hybrid&w=740"
    password = Encryption.hash_password('cpfcadt')

    return name, phone, reg_date_str, login_date_str, img_path, password


# Connect to the database
try:
    connection = mysql.connector.connect(
        host=DB_HOST, user=DB_USER, password=DB_PASSWORD, database=DB_NAME
    )
    cursor = connection.cursor()
except mysql.connector.Error as err:
    print("Error connecting to database:", err)
    exit()

# Insert customer data
print('Start inserting customer records ...')
num_customers = 140000
batch_size = 1000
rounds = num_customers // batch_size

for i in range(rounds):
    for _ in range(batch_size):
        name, phone, reg_date, login_date, img_path, password = generate_customer_data()
        insert_query = """
        INSERT INTO customer (name, phone_number, registration_date, login_date, profile_img_path, password)
        VALUES (%s, %s, %s, %s, %s, %s)
        """
        try:
            cursor.execute(insert_query, (name, phone, reg_date, login_date, img_path, password))
        except mysql.connector.Error as err:
            print(f"Error inserting record: {err}")
    connection.commit()
    print(f'Inserted batch {i+1}/{rounds} ({batch_size} records)')

print(f"Successfully inserted {num_customers} records into the customer table.")

cursor.close()
connection.close()
print('Finished inserting!!!')


# product 

In [3]:
import random
from datetime import datetime, timedelta
import mysql.connector

# --- Database connection details ---
DB_HOST = "localhost"
DB_USER = "root"
DB_PASSWORD = "1234"
DB_NAME = "computer_shop"

# --- Lookup Data (unchanged) ---
BRANDS = {
    6: 'AMD', 2: 'ASUS', 12: 'Corsair', 14: 'Crucial', 16: 'Dell',
    13: 'Gigabyte', 1: 'InnovateTech', 5: 'Intel', 15: 'Kingston',
    10: 'Logitech', 4: 'MSI', 3: 'NVIDIA', 11: 'Razer', 7: 'Samsung',
    9: 'Seagate', 8: 'Western Digital'
}
CATEGORIES = {1: 'Accessories', 2: 'Computers', 3: 'Gaming', 4: 'Hardware Components', 5: 'Networking', 6: 'Office Equipment', 7: 'Printers & Scanners', 8: 'Software', 9: 'Storage Devices'}
TYPES = {1: 'CPU', 2: 'RAM', 3: 'VGA', 4: 'Motherboard', 5: 'SSD', 6: 'HDD', 7: 'Power Supply Unit', 8: 'CPU Cooler', 9: 'Case', 10: 'Monitor', 11: 'Keyboard', 12: 'Mouse', 13: 'Printer', 14: 'Router', 15: 'Labtop', 16: 'Operating System'}
TYPE_CATEGORY_MAP = {1: 4, 2: 4, 3: 4, 4: 4, 5: 4, 6: 4, 7: 4, 8: 4, 9: 4, 10: 1, 11: 1, 12: 1, 13: 7, 14: 5, 15: 2, 16: 8}
TYPE_BRAND_SUGGESTIONS = {1: [5, 6], 2: [12, 14, 15], 3: [3, 6, 2, 13, 4], 4: [2, 13, 4], 5: [7, 9, 8, 14, 15], 6: [9, 8], 7: [12, 2, 4], 8: [12, 2, 4], 9: [12, 2, 4], 10: [7, 16, 2, 4, 11], 11: [10, 11, 12, 16], 12: [10, 11, 12, 16], 13: [16], 14: [2], 15: [16, 2, 4, 13, 11], 16: [1]}

# --- UPDATED: Function to get existing product counters ---
def get_existing_max_product_ids(cursor):
    """
    Parses existing product_codes to find the highest counter for each type.
    This new version assumes a fixed-width format: TTNNNNN (e.g., 0100001)
    """
    max_ids = {type_id: 0 for type_id in TYPES.keys()}
    
    try:
        # We only need the product_code, as the type is embedded in it.
        cursor.execute("SELECT product_code FROM product")
        for (product_code_db,) in cursor: # Unpack the tuple from fetch
            try:
                # New format assumes first 2 chars are type_id, rest is the counter
                # Increase total length check if counter padding is increased
                if len(product_code_db) >= 7:
                    type_id = int(product_code_db[0:2])
                    counter = int(product_code_db[2:])
                    
                    if type_id in max_ids and counter > max_ids[type_id]:
                        max_ids[type_id] = counter
            except (ValueError, IndexError):
                # This will skip any product_code that doesn't match the new format
                print(f"Warning: Could not parse product_code '{product_code_db}'. Skipping.")
    except mysql.connector.Error as err:
        print(f"Error fetching existing product IDs: {err}")
        return {type_id: 1 for type_id in TYPES.keys()}
        
    # Increment each max_id by 1 to get the next available counter.
    return {tid: val + 1 for tid, val in max_ids.items()}

# --- UPDATED: Product Data Generation Function ---
def generate_products_data(num_products_per_type=500, initial_product_counters=None):
    products_data = []
    product_counters = initial_product_counters.copy() if initial_product_counters is not None else {type_id: 1 for type_id in TYPES.keys()}

    for type_id, type_name in TYPES.items():
        category_id = TYPE_CATEGORY_MAP.get(type_id)
        if not category_id:
            print(f"Warning: No category for Type ID {type_id}. Skipping.")
            continue
        
        category_title = CATEGORIES[category_id]
        available_brand_ids = TYPE_BRAND_SUGGESTIONS.get(type_id, list(BRANDS.keys()))
        if not available_brand_ids:
            available_brand_ids = list(BRANDS.keys())

        for _ in range(num_products_per_type):
            product_id_in_type = product_counters[type_id]
            
            # --- MODIFIED LINE ---
            # Create a non-ambiguous product_code by padding both type and counter.
            # Format: TTNNNNN (Type-Type-Counter-Counter-Counter-Counter-Counter)
            # e.g., type 1, counter 1 -> "0100001"; type 15, counter 123 -> "1500123"
            product_code = f"{str(type_id).zfill(2)}{str(product_id_in_type).zfill(5)}"
            
            product_counters[type_id] += 1

            brand_id = random.choice(available_brand_ids)
            brand_name = BRANDS[brand_id]
            name = f"{category_title} {type_name} {brand_name} {product_code}"
            description = f"A high-performance {type_name} from {brand_name}. Designed for {category_title.lower()} needs. Product code: {product_code}."
            price = round(random.uniform(50.00, 3000.00), 2)
            stock_quantity = random.randint(50, 3000)
            image_path = f"/images/{type_name.lower().replace(' ', '_')}_{product_code}.jpg"
            is_active = random.choice([True, False])
            
            start_date = datetime.now() - timedelta(days=365)
            end_date = datetime.now()
            random_days = random.randint(0, (end_date - start_date).days)
            last_restock_date = (start_date + timedelta(days=random_days)).strftime('%Y-%m-%d %H:%M:%S')

            products_data.append((
                product_code, name, description, price, stock_quantity,
                image_path, brand_id, is_active, last_restock_date,
                category_id, type_id
            ))
    return products_data

# --- Main execution block (unchanged) ---
if __name__ == "__main__":
    connection = None
    cursor = None
    try:
        connection = mysql.connector.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, database=DB_NAME)
        cursor = connection.cursor()

        initial_product_counters = get_existing_max_product_ids(cursor)
        print("Initial product counters based on existing data:", initial_product_counters)

        products_to_insert = generate_products_data(
            num_products_per_type=50, 
            initial_product_counters=initial_product_counters
        )
        print(f"Generated {len(products_to_insert)} new product records.")

        insert_query = """
        INSERT INTO product (product_code, name, description, price, stock_quantity, 
                             image_path, brand_id, is_active, last_restock_date, 
                             category_id, type_id)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        batch_size = 500
        num_products = len(products_to_insert)
        print(f'Start inserting {num_products} product records ...')

        for i in range(0, num_products, batch_size):
            batch = products_to_insert[i:i + batch_size]
            try:
                cursor.executemany(insert_query, batch)
                connection.commit()
                print(f'Inserted batch {i // batch_size + 1}/{(num_products + batch_size - 1) // batch_size} ({len(batch)} records)')
            except mysql.connector.Error as err:
                print(f"Error inserting batch starting at index {i}: {err}")
                connection.rollback()
                break

        print(f"Successfully processed {num_products} records for the product table.")

    except mysql.connector.Error as err:
        print("Error connecting to database or during operation:", err)
    finally:
        if cursor:
            cursor.close()
        if connection and connection.is_connected():
            connection.close()
        print('Finished inserting products!!!')

Initial product counters based on existing data: {1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1}
Generated 800 new product records.
Start inserting 800 product records ...
Inserted batch 1/2 (500 records)
Inserted batch 2/2 (300 records)
Successfully processed 800 records for the product table.
Finished inserting products!!!


# Insert Address

In [ ]:
import random
import string
import mysql.connector
import json
import os

# --- Configuration ---

# Database connection details
DB_HOST = "localhost"
DB_USER = "root"
DB_PASSWORD = "1234"
DB_NAME = "computer_shop"

# Path to the Cambodia provinces/districts JSON file
CAMBODIA_JSON_FILE = 'cambodia.json'

# --- Functions ---

def load_cambodia_data(filepath):
    """Loads province/district data from the specified JSON file."""
    if not os.path.exists(filepath):
        print(f"Error: {filepath} not found. Please ensure the JSON file is in the same directory.")
        return None
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            return json.load(f)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from {filepath}: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred while reading {filepath}: {e}")
        return None

# --- Main script execution ---

connection = None
try:
    # Establish the database connection
    connection = mysql.connector.connect(
        host=DB_HOST, user=DB_USER, password=DB_PASSWORD, database=DB_NAME
    )
    cursor = connection.cursor()

    # 1. Get ALL customer IDs from the customer table.
    print("Step 1: Fetching all customer IDs...")
    cursor.execute("SELECT customer_id FROM customer")
    # Using a set for efficient lookup and difference calculation
    all_customer_ids = {item[0] for item in cursor.fetchall()} # type: ignore
    print(f"Found {len(all_customer_ids)} total customers.")

    # 2. Get IDs of customers who ALREADY have an address.
    print("Step 2: Fetching IDs of customers who already have an address...")
    cursor.execute("SELECT customer_id FROM address")
    # Also using a set
    ids_with_address = {item[0] for item in cursor.fetchall()} # type: ignore
    print(f"Found {len(ids_with_address)} customers with an existing address.")

    # 3. Calculate which customer IDs need an address by finding the difference.
    # This is the core of the new concept.
    ids_to_insert_set = all_customer_ids - ids_with_address
    customer_ids_to_insert = list(ids_to_insert_set) # Convert back to a list for iteration
    
    num_to_insert = len(customer_ids_to_insert)

    if num_to_insert == 0:
        print("All customers already have an address. Nothing to do.")
        connection.close()
        exit() # Exit the script cleanly
    
    print(f"Step 3: Calculated that {num_to_insert} new addresses need to be inserted.")

    # 4. Load Cambodia province/district data
    cambodia_data = load_cambodia_data(CAMBODIA_JSON_FILE)
    if not cambodia_data:
        print("Aborting address insertion due to missing or invalid Cambodia data.")
        connection.close()
        exit()

    # 5. Prepare and insert address data for the needed customers
    print("Starting address insertion...")
    batch_size = 1000  # How many addresses to insert in one go
    address_data_batch = []

    # Iterate through the list of customer IDs that need an address
    for i, customer_id in enumerate(customer_ids_to_insert, 1):
        # Randomly select a province
        random_province = random.choice(cambodia_data)
        province_name = random_province['province_name']

        # Randomly select a district from the chosen province
        if random_province.get('districts'): # Use .get() for safety
            random_district = random.choice(random_province['districts'])
            district_name = random_district.get('name') # Use .get() for safety
        else:
            district_name = "N/A" # Fallback

        # Generate a simple random street line
        street_line = f"{random.randint(1, 200)} {random.choice(['Street', 'Road', 'Blvd'])} {random.choice(string.ascii_uppercase)}{random.randint(100, 999)}"

        # Set commune and google_map_link to None as requested
        commune = None
        google_map_link = None

        address_data_batch.append((customer_id, street_line, commune, district_name, province_name, google_map_link))

        # Execute batch insert when the batch is full or it's the last item
        if (i % batch_size == 0) or (i == num_to_insert):
            insert_address_query = """
            INSERT INTO address (customer_id, street_line, commune, district, province, google_map_link)
            VALUES (%s, %s, %s, %s, %s, %s)
            """
            try:
                cursor.executemany(insert_address_query, address_data_batch)
                connection.commit()
                print(f'-> Inserted batch. Progress: {i}/{num_to_insert} address records.')
            except mysql.connector.Error as err:
                print(f"Error inserting address batch: {err}")
                connection.rollback()
            address_data_batch = []  # Clear the batch for the next set of records

    print(f"Successfully inserted {num_to_insert} new addresses into the address table.")

except mysql.connector.Error as err:
    print("Database error:", err)
finally:
    if connection and connection.is_connected():
        cursor.close() # type: ignore
        connection.close()
        print('Database connection closed.')

print('Finished inserting addresses!!!')

# Insert Order and then Order Item

In [ ]:
import random
import mysql.connector
from decimal import Decimal
import time
import sys

# --- Configuration ---
DB_HOST = "localhost"
DB_USER = "root"
DB_PASSWORD = "1234"
DB_NAME = "computer_shop"

# --- GOALS ---
TOTAL_ORDERS_TO_CREATE = 3_000_000
# How many orders to generate and insert in a single batch.
# Adjust based on your system's RAM. 10,000 is a good starting point.
ORDER_BATCH_SIZE = 10_000 
# Min/max number of items per order. An average of 3.5 will give ~10.5M order items.
MIN_ITEMS_PER_ORDER = 2
MAX_ITEMS_PER_ORDER = 5

def print_progress(iteration, total, start_time, prefix='', suffix='', length=50, fill='█'):
    """A simple helper function to show a progress bar."""
    elapsed_time = time.time() - start_time
    percent = ("{0:.1f}").format(100 * (iteration / float(total)))
    filled_length = int(length * iteration // total)
    bar = fill * filled_length + '-' * (length - filled_length)
    # Estimate remaining time
    if iteration > 0:
        time_per_iteration = elapsed_time / iteration
        remaining_time = time_per_iteration * (total - iteration)
        eta = time.strftime("%H:%M:%S", time.gmtime(remaining_time))
    else:
        eta = "N/A"
    
    sys.stdout.write(f'\r{prefix} |{bar}| {percent}% {suffix} (ETA: {eta})')
    sys.stdout.flush()

def calculate_price_at_purchase(base_price, discount_type, discount_value):
    """Calculates the final price after applying a discount."""
    if discount_value is None:
        return base_price
        
    if discount_type == 'Percentage':
        return base_price * (1 - (discount_value / 100))
    elif discount_type == 'Fixed':
        return max(Decimal('0.00'), base_price - discount_value)
    return base_price

# --- Main Execution ---
if __name__ == "__main__":
    connection = None
    print("Starting high-performance data generation script.")
    print(f"Goal: {TOTAL_ORDERS_TO_CREATE:,} orders and ~{TOTAL_ORDERS_TO_CREATE * ((MIN_ITEMS_PER_ORDER + MAX_ITEMS_PER_ORDER)/2):,.0f} order items.")

    try:
        # Establish connection with manual transaction control
        connection = mysql.connector.connect(
            host=DB_HOST, user=DB_USER, password=DB_PASSWORD, database=DB_NAME
        )
        # We will manually commit after each batch
        connection.autocommit = False  # type: ignore
        cursor = connection.cursor()

        # --- PRE-LOADING PHASE: Load necessary data into memory ---
        print("\nStep 1: Pre-loading data into memory... (this may take a moment)")

        # 1a. Load all customer and address data
        cursor.execute("""
            SELECT customer_id, 
                (SELECT address_id FROM address WHERE customer_id = c.customer_id LIMIT 1) as address_id
            FROM customer c 
            HAVING address_id IS NOT NULL
        """)
        customer_address_pairs = cursor.fetchall()
        if not customer_address_pairs:
            raise Exception("No customers with addresses found. Cannot proceed.")
        print(f"-> Loaded {len(customer_address_pairs):,} customer/address pairs.")

        # 1b. Load all product data (code, price, stock, discount)
        # We need the real stock from `product`, not just the view
        cursor.execute("""
            SELECT
                p.product_code, p.price, p.stock_quantity,
                spd.discount_type, spd.discount_value
            FROM product AS p
            LEFT JOIN show_product_discount AS spd ON spd.product_code = p.product_code
            WHERE p.stock_quantity > 0
        """)
        # Store products in a list of dicts for easy access and modification
        # Convert Decimal to float for performance, reconvert on insert
        all_products = [
            {
                'code': row[0],
                'price': Decimal(row[1]),
                'stock': row[2],
                'discount_type': row[3],
                'discount_value': Decimal(row[4]) if row[4] is not None else None
            }
            for row in cursor.fetchall()
        ]
        if not all_products:
            raise Exception("No products with stock > 0 found. Cannot proceed.")
        print(f"-> Loaded {len(all_products):,} in-stock products.")

        # --- BATCH PROCESSING PHASE ---
        print("\nStep 2: Starting batch generation and insertion...\n")
        
        # Temporarily disable foreign key checks for massive speedup
        cursor.execute("SET foreign_key_checks = 0;")
        
        total_items_generated = 0
        num_batches = TOTAL_ORDERS_TO_CREATE // ORDER_BATCH_SIZE
        script_start_time = time.time()

        for i in range(num_batches):
            batch_start_time = time.time()
            print_progress(i, num_batches, script_start_time, prefix=f'Batch {i+1}/{num_batches}:', suffix='Complete')

            orders_batch = []
            order_items_batch = []
            stock_updates = {} # {product_code: total_qty_to_reduce}

            # Generate order data for the batch IN MEMORY
            for _ in range(ORDER_BATCH_SIZE):
                customer_id, address_id = random.choice(customer_address_pairs)
                # Note: We don't need the date, it has a DEFAULT. Status is also defaulted.
                orders_batch.append((customer_id, address_id, random.choice(['Pending', 'Processing', 'Delivered'])))
            
            # INSERT the batch of orders
            insert_order_query = "INSERT INTO orders (customer_id, address_id, order_status) VALUES (%s, %s, %s)"
            cursor.executemany(insert_order_query, orders_batch)
            
            # IMPORTANT: Get the ID of the FIRST order inserted in this batch
            first_order_id = cursor.lastrowid
            last_order_id = first_order_id + ORDER_BATCH_SIZE - 1

            # Now, generate order_items for the orders we just created
            for order_id in range(first_order_id, last_order_id + 1):
                num_items = random.randint(MIN_ITEMS_PER_ORDER, MAX_ITEMS_PER_ORDER)
                
                # To avoid picking the same product twice for one order
                products_chosen_for_this_order = set()

                for _ in range(num_items):
                    # Try to find a valid product with stock
                    for _ in range(10): # Try 10 times to find a unique, in-stock product
                        product = random.choice(all_products)
                        # Check if product is in stock (in our memory copy) AND not already in this order
                        if product['stock'] > 0 and product['code'] not in products_chosen_for_this_order:
                            break
                    else:
                        continue # Couldn't find a suitable product, skip adding this item

                    qty = 1 # Keep it simple with qty=1 for mass generation
                    price = calculate_price_at_purchase(product['price'], product['discount_type'], product['discount_value'])
                    
                    order_items_batch.append((order_id, product['code'], qty, price))
                    total_items_generated += 1
                    products_chosen_for_this_order.add(product['code'])

                    # Decrement the stock in our Python list (in-memory)
                    product['stock'] -= qty
                    # Aggregate the total stock reduction for this batch
                    stock_updates[product['code']] = stock_updates.get(product['code'], 0) + qty
            
            # INSERT the batch of order items
            if order_items_batch:
                insert_order_item_query = "INSERT INTO orderitem (order_id, product_code, qty, price_at_purchase) VALUES (%s, %s, %s, %s)"
                cursor.executemany(insert_order_item_query, order_items_batch)

            # UPDATE the stock for all affected products in one go
            if stock_updates:
                # Build a single powerful UPDATE statement
                # UPDATE product SET stock_quantity = CASE product_code
                #   WHEN 'code1' THEN stock_quantity - 5
                #   WHEN 'code2' THEN stock_quantity - 12
                # END
                # WHERE product_code IN ('code1', 'code2')
                sql_case_parts = []
                product_codes_to_update = []
                for code, qty_to_reduce in stock_updates.items():
                    sql_case_parts.append(f"WHEN %s THEN stock_quantity - %s")
                    product_codes_to_update.extend([code, qty_to_reduce])
                
                # Add the product codes for the WHERE clause at the end
                where_clause_placeholders = ', '.join(['%s'] * len(stock_updates))
                product_codes_to_update.extend(stock_updates.keys())

                update_query = f"""
                    UPDATE product
                    SET stock_quantity = CASE product_code {' '.join(sql_case_parts)} END
                    WHERE product_code IN ({where_clause_placeholders})
                """
                cursor.execute(update_query, tuple(product_codes_to_update))

            # Commit the entire batch transaction (orders, items, stock updates)
            connection.commit()

        # Final progress bar update to show 100%
        print_progress(num_batches, num_batches, script_start_time, prefix=f'Batch {num_batches}/{num_batches}:', suffix='Complete')
        
        # Re-enable foreign key checks
        cursor.execute("SET foreign_key_checks = 1;")
        connection.commit()
        
        total_time = time.time() - script_start_time
        print(f"\n\n--- Generation Complete ---")
        print(f"Successfully created {TOTAL_ORDERS_TO_CREATE:,} orders.")
        print(f"Successfully created {total_items_generated:,} order items.")
        print(f"Total time: {total_time:.2f} seconds.")

    except mysql.connector.Error as err:
        print(f"\nDATABASE ERROR: {err}")
        if connection:
            print("-> Rolling back last transaction...")
            connection.rollback()
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")
    finally:
        if connection and connection.is_connected():
            # Make sure foreign keys are re-enabled in case of an error
            try:
                cursor.execute("SET foreign_key_checks = 1;")
                connection.commit()
            except:
                pass # Ignore errors here as we are already in a finally block
            cursor.close()
            connection.close()
            print("Database connection closed.")

## inset payment transaction

In [ ]:
import random
import mysql.connector
import time
import sys
from decimal import Decimal

# --- Configuration ---
DB_HOST = "localhost"
DB_USER = "root"
DB_PASSWORD = "1234"  # IMPORTANT: Replace with your actual database password
DB_NAME = "computer_shop"

# --- GOALS ---
# How many records to insert in a single batch.
BATCH_SIZE = 10_000

# --- Status Probabilities ---
# 98% Completed, 0.001% Pending, rest is Failed
STATUS_CHOICES = ['Completed', 'Pending', 'Failed']
STATUS_WEIGHTS = [99.0, 0.001, 0.999] 

def print_progress(iteration, total, start_time, prefix='', suffix='', length=50, fill='█'):
    """A simple helper function to show a progress bar."""
    # This check is to handle if total is zero
    if total == 0:
        total = 1
        
    elapsed_time = time.time() - start_time
    percent = ("{0:.1f}").format(100 * (iteration / float(total)))
    filled_length = int(length * iteration // total)
    bar = fill * filled_length + '-' * (length - filled_length)
    if iteration > 0:
        time_per_iteration = elapsed_time / iteration
        remaining_time = time_per_iteration * (total - iteration)
        eta = time.strftime("%H:%M:%S", time.gmtime(remaining_time))
    else:
        eta = "N/A"
    
    sys.stdout.write(f'\r{prefix} |{bar}| {percent}% {suffix} (ETA: {eta})')
    sys.stdout.flush()

# --- Main Execution ---
if __name__ == "__main__":
    connection = None
    print("Starting Payment Transaction generation script.")

    try:
        connection = mysql.connector.connect(
            host=DB_HOST, user=DB_USER, password=DB_PASSWORD, database=DB_NAME
        )
        cursor = connection.cursor()

        # --- PRE-LOADING PHASE: Load necessary data into memory ---
        print("\nStep 1: Pre-loading data from database...")

        # 1a. Get all enabled Payment Method IDs
        cursor.execute("SELECT pay_method_id FROM paymentmethod WHERE is_enable = 1")
        all_payment_method_ids = [item[0] for item in cursor.fetchall()]
        if not all_payment_method_ids:
            raise Exception("No enabled payment methods found. Please add data to 'paymentmethod' table.")
        print(f"-> Loaded {len(all_payment_method_ids):,} enabled payment methods.")

        # 1b. Get all Order IDs that do not already have a payment transaction
        cursor.execute("""
            SELECT o.order_id 
            FROM orders o
            LEFT JOIN paymenttransaction pt ON o.order_id = pt.order_id
            WHERE pt.transaction_id IS NULL
        """)
        order_ids_to_process = [item[0] for item in cursor.fetchall()]
        if not order_ids_to_process:
            print("\nAll existing orders already have a payment transaction. No new data to generate.")
            sys.exit()
        print(f"-> Found {len(order_ids_to_process):,} orders needing a payment transaction.")
        
        # --- DATA GENERATION & CALCULATION PHASE ---
        print("\nStep 2: Calculating order totals and generating transaction data...")
        
        transactions_to_insert = []
        script_start_time = time.time()
        
        # This is an efficient way to get all totals at once
        # Create a placeholder string for the IN clause: %s, %s, %s ...
        placeholders = ', '.join(['%s'] * len(order_ids_to_process))
        query = f"SELECT order_id, calculate_order_total(order_id) FROM orders WHERE order_id IN ({placeholders})"
        
        cursor.execute(query, tuple(order_ids_to_process))
        
        order_totals = cursor.fetchall()

        for order_id, total_amount in order_totals:
            # Randomly choose a payment method
            pay_method_id = random.choice(all_payment_method_ids)
            
            # Determine status based on weights
            status = random.choices(STATUS_CHOICES, weights=STATUS_WEIGHTS, k=1)[0]
            
            # Append the complete transaction record for insertion
            transactions_to_insert.append((
                order_id,
                pay_method_id,
                total_amount, # The amount calculated by the SQL function
                status
            ))

        print(f"-> Successfully generated {len(transactions_to_insert):,} new transaction records.")

        # --- BATCH INSERTION PHASE ---
        if transactions_to_insert:
            print("\nStep 3: Inserting data into the database in batches...")
            
            insert_query = """
                INSERT INTO paymenttransaction (order_id, pay_method_id, amount, status) 
                VALUES (%s, %s, %s, %s)
            """
            
            total_inserted = 0
            for i in range(0, len(transactions_to_insert), BATCH_SIZE):
                batch = transactions_to_insert[i:i + BATCH_SIZE]
                cursor.executemany(insert_query, batch)
                connection.commit()
                total_inserted += len(batch)
                print_progress(total_inserted, len(transactions_to_insert), script_start_time, prefix='Inserting:', suffix='Complete')

        total_time = time.time() - script_start_time
        print(f"\n\n--- Generation Complete ---")
        print(f"Successfully inserted {len(transactions_to_insert):,} records into 'paymenttransaction'.")
        print(f"Total time: {total_time:.2f} seconds.")

    except mysql.connector.Error as err:
        print(f"\nDATABASE ERROR: {err}")
        if 'calculate_order_total' in str(err) and 'does not exist' in str(err):
            print("\nCRITICAL: The function 'calculate_order_total' was not found in the database.")
            print("Please create the function using the script from the Canvas before running this script.")
        elif connection:
            print("-> Rolling back the last transaction...")
            connection.rollback()
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print("Database connection closed.")


## product promotion

In [ ]:
import random
import mysql.connector
import time
import sys

# --- Configuration ---
DB_HOST = "localhost"
DB_USER = "root"
DB_PASSWORD = "1234"  # Replace with your database password
DB_NAME = "computer_shop"

# --- GOALS ---
# Total number of product-promotion links to create
TOTAL_PROMOTIONS_TO_CREATE = 3000
# How many records to insert in a single batch.
# For this task, we can generate all and insert once.
BATCH_SIZE = 1000

def print_progress(iteration, total, start_time, prefix='', suffix='', length=50, fill='█'):
    """A simple helper function to show a progress bar."""
    elapsed_time = time.time() - start_time
    percent = ("{0:.1f}").format(100 * (iteration / float(total)))
    filled_length = int(length * iteration // total)
    bar = fill * filled_length + '-' * (length - filled_length)
    # Estimate remaining time
    if iteration > 0:
        time_per_iteration = elapsed_time / iteration
        remaining_time = time_per_iteration * (total - iteration)
        eta = time.strftime("%H:%M:%S", time.gmtime(remaining_time))
    else:
        eta = "N/A"
    
    sys.stdout.write(f'\r{prefix} |{bar}| {percent}% {suffix} (ETA: {eta})')
    sys.stdout.flush()

# --- Main Execution ---
if __name__ == "__main__":
    connection = None
    print("Starting productpromotion data generation script.")
    print(f"Goal: Create {TOTAL_PROMOTIONS_TO_CREATE:,} product-promotion links.")

    try:
        # Establish connection with the database
        connection = mysql.connector.connect(
            host=DB_HOST, user=DB_USER, password=DB_PASSWORD, database=DB_NAME
        )
        cursor = connection.cursor()

        # --- PRE-LOADING PHASE: Load necessary data into memory ---
        print("\nStep 1: Pre-loading existing products and promotions...")

        # 1a. Load all product codes
        cursor.execute("SELECT product_code FROM product")
        all_product_codes = [item[0] for item in cursor.fetchall()]
        if not all_product_codes:
            raise Exception("No products found in the 'product' table. Please add products first.")
        print(f"-> Loaded {len(all_product_codes):,} product codes.")

        # 1b. Load all promotion IDs
        cursor.execute("SELECT promotion_id FROM promotion")
        all_promotion_ids = [item[0] for item in cursor.fetchall()]
        if not all_promotion_ids:
            raise Exception("No promotions found in the 'promotion' table. Please add promotions first.")
        print(f"-> Loaded {len(all_promotion_ids):,} promotion IDs.")
        
        # 1c. Load existing productpromotion links to avoid duplicates
        cursor.execute("SELECT product_code, promotion_id FROM productpromotion")
        existing_links = set(cursor.fetchall())
        print(f"-> Loaded {len(existing_links):,} existing links to prevent duplicates.")


        # --- DATA GENERATION PHASE ---
        print("\nStep 2: Generating new product-promotion links...")
        
        productpromotions_to_insert = []
        generated_links = set() # Use a set for fast checking of uniqueness
        script_start_time = time.time()

        while len(productpromotions_to_insert) < TOTAL_PROMOTIONS_TO_CREATE:
            product_code = random.choice(all_product_codes)
            promotion_id = random.choice(all_promotion_ids)
            
            # Create a tuple for the new link
            new_link = (product_code, promotion_id)
            
            # Check if the link already exists in the DB or has been generated in this session
            if new_link not in existing_links and new_link not in generated_links:
                generated_links.add(new_link)
                productpromotions_to_insert.append(new_link)

            # Safety break to prevent infinite loops if all possible combinations are exhausted
            if len(generated_links) + len(existing_links) >= len(all_product_codes) * len(all_promotion_ids):
                print("\nWarning: All possible product-promotion combinations have been created.")
                break
        
        print(f"-> Successfully generated {len(productpromotions_to_insert):,} new unique links.")

        # --- BATCH INSERTION PHASE ---
        if productpromotions_to_insert:
            print("\nStep 3: Inserting data into the database...")
            
            insert_query = "INSERT INTO productpromotion (product_code, promotion_id) VALUES (%s, %s)"
            
            # Split data into batches
            total_inserted = 0
            for i in range(0, len(productpromotions_to_insert), BATCH_SIZE):
                batch = productpromotions_to_insert[i:i + BATCH_SIZE]
                cursor.executemany(insert_query, batch)
                connection.commit()
                total_inserted += len(batch)
                print_progress(total_inserted, len(productpromotions_to_insert), script_start_time, prefix='Inserting:', suffix='Complete')

        total_time = time.time() - script_start_time
        print(f"\n\n--- Generation Complete ---")
        print(f"Successfully inserted {len(productpromotions_to_insert):,} records into 'productpromotion'.")
        print(f"Total time: {total_time:.2f} seconds.")

    except mysql.connector.Error as err:
        print(f"\nDATABASE ERROR: {err}")
        if connection:
            print("-> Rolling back transaction...")
            connection.rollback()
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print("Database connection closed.")


## feedback for product generator

In [ ]:
import random
import mysql.connector
from datetime import datetime
import time
import sys

# --- Configuration ---
DB_HOST = "localhost"
DB_USER = "root"
DB_PASSWORD = "1234"  # IMPORTANT: Replace with your actual database password
DB_NAME = "computer_shop"

# --- GOALS ---
TOTAL_FEEDBACK_TO_CREATE = 80_000
# How many records to generate and insert in a single batch.
# Adjust based on your system's RAM. 10,000 is a good starting point.
BATCH_SIZE = 10_000

# --- Sample Comments based on Rating ---
COMMENTS_POSITIVE = [
    "Absolutely fantastic product! Exceeded all my expectations. Highly recommended.",
    "Works perfectly, just as described. Very happy with this purchase.",
    "Excellent build quality and performance. Worth every penny.",
    "I'm really impressed with this. It has made a huge difference.",
    "Five stars! The quality is top-notch and it arrived very quickly."
]
COMMENTS_NEUTRAL = [
    "It's an okay product. It does the job, but nothing special.",
    "The product is decent for the price. No major complaints.",
    "Works as expected. Met my basic needs.",
    "Average quality. You get what you pay for.",
    "It's fine. Not great, not terrible."
]
COMMENTS_NEGATIVE = [
    "Very disappointed with this purchase. It broke after just a few uses.",
    "Did not work as advertised. The description is misleading.",
    "Poor quality and feels cheap. I would not recommend this to anyone.",
    "Arrived damaged and was a hassle to return. A terrible experience.",
    "This is not worth the money. Save yourself the trouble and buy something else."
]

def print_progress(iteration, total, start_time, prefix='', suffix='', length=50, fill='█'):
    """A simple helper function to show a progress bar."""
    elapsed_time = time.time() - start_time
    percent = ("{0:.1f}").format(100 * (iteration / float(total)))
    filled_length = int(length * iteration // total)
    bar = fill * filled_length + '-' * (length - filled_length)
    if iteration > 0:
        time_per_iteration = elapsed_time / iteration
        remaining_time = time_per_iteration * (total - iteration)
        eta = time.strftime("%H:%M:%S", time.gmtime(remaining_time))
    else:
        eta = "N/A"
    
    sys.stdout.write(f'\r{prefix} |{bar}| {percent}% {suffix} (ETA: {eta})')
    sys.stdout.flush()

# --- Main Execution ---
if __name__ == "__main__":
    connection = None
    print("Starting product feedback data generation script.")
    print(f"Goal: {TOTAL_FEEDBACK_TO_CREATE:,} unique feedback records.")

    try:
        connection = mysql.connector.connect(
            host=DB_HOST, user=DB_USER, password=DB_PASSWORD, database=DB_NAME
        )
        cursor = connection.cursor()

        # --- PRE-LOADING PHASE: Load necessary data into memory ---
        print("\nStep 1: Pre-loading data into memory...")

        # 1a. Load all customer IDs
        cursor.execute("SELECT customer_id FROM customer")
        all_customer_ids = [item[0] for item in cursor.fetchall()]
        if not all_customer_ids:
            raise Exception("No customers found. Please add customer data first.")
        print(f"-> Loaded {len(all_customer_ids):,} customer IDs.")

        # 1b. Load all product codes
        cursor.execute("SELECT product_code FROM product")
        all_product_codes = [item[0] for item in cursor.fetchall()]
        if not all_product_codes:
            raise Exception("No products found. Please add product data first.")
        print(f"-> Loaded {len(all_product_codes):,} product codes.")

        # 1c. Load existing feedback to prevent creating duplicates
        cursor.execute("SELECT customer_id, product_code FROM productfeedback")
        existing_feedback = set(cursor.fetchall())
        print(f"-> Loaded {len(existing_feedback):,} existing feedback records to avoid duplicates.")

        # --- DATA GENERATION PHASE ---
        print("\nStep 2: Generating new unique feedback data...")
        
        feedback_to_insert = []
        generated_this_session = set()
        script_start_time = time.time()

        # Generate unique feedback records in memory
        while len(feedback_to_insert) < TOTAL_FEEDBACK_TO_CREATE:
            customer_id = random.choice(all_customer_ids)
            product_code = random.choice(all_product_codes)
            
            feedback_pair = (customer_id, product_code)
            
            # CRITICAL CHECK: Ensure the customer hasn't already reviewed this product
            if feedback_pair in existing_feedback or feedback_pair in generated_this_session:
                continue

            # Generate rating and a corresponding comment
            rating = random.randint(1, 5)
            if rating >= 4:
                comment = random.choice(COMMENTS_POSITIVE)
            elif rating == 3:
                comment = random.choice(COMMENTS_NEUTRAL)
            else:
                comment = random.choice(COMMENTS_NEGATIVE)
            
            feedback_date = datetime.now()

            feedback_to_insert.append((customer_id, product_code, rating, comment, feedback_date))
            generated_this_session.add(feedback_pair)

            # Safety break to prevent an infinite loop if all combinations are exhausted
            if len(existing_feedback) + len(generated_this_session) >= len(all_customer_ids) * len(all_product_codes):
                print("\nWarning: All possible unique feedback combinations have been generated.")
                break

        print(f"-> Successfully generated {len(feedback_to_insert):,} new feedback records.")

        # --- BATCH INSERTION PHASE ---
        if feedback_to_insert:
            print("\nStep 3: Inserting data into the database in batches...")
            
            insert_query = """
                INSERT INTO productfeedback (customer_id, product_code, rating, comment, feedback_date) 
                VALUES (%s, %s, %s, %s, %s)
            """
            
            total_inserted = 0
            for i in range(0, len(feedback_to_insert), BATCH_SIZE):
                batch = feedback_to_insert[i:i + BATCH_SIZE]
                cursor.executemany(insert_query, batch)
                connection.commit()
                total_inserted += len(batch)
                print_progress(total_inserted, len(feedback_to_insert), script_start_time, prefix='Inserting:', suffix='Complete')

        total_time = time.time() - script_start_time
        print(f"\n\n--- Generation Complete ---")
        print(f"Successfully inserted {len(feedback_to_insert):,} records into 'productfeedback'.")
        print(f"Total time: {total_time:.2f} seconds.")

    except mysql.connector.Error as err:
        print(f"\nDATABASE ERROR: {err}")
        if connection:
            print("-> Rolling back the last transaction...")
            connection.rollback()
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print("Database connection closed.")